In [ ]:
'''
To use this data processor, run all cells in order. First, a pre-defined remote data source 
with gobal information on Covid-19 including number of cases, recoveries, and deaths, will
be retrieved and stored locally in a JSON format in the same folder this notebook is in.
Then, when asked for an input, the user must input 'CSV' or 'SQL' and the data source will
be converted to a CSV file or SQL Database file, which will also be stored locally. 
Finally, a summary of the data file including number of rows and columns is provided.
'''

In [133]:
import os
import json
import pprint
import requests
import requests.exceptions
import pandas as pd
from sqlalchemy import create_engine

In [134]:
### Pre-defined data source
base_url = "https://coronavirus.m.pipedream.net/"

In [135]:
def get_api_response(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
    
    except requests.exceptions.HTTPError as errh:
        return "An Http Error occurred: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
        return "An Error Connecting to the API occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "A Timeout Error occurred: " + repr(errt)
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)

    result = json.dumps(response.json(), sort_keys=True, indent=4)

    return result

### Retrieve remote data file by URL using the requests http library
json_string = get_api_response(base_url)

In [136]:
### Write data to a local json file using the json package
with open('Covid.json', 'w',encoding='utf8') as file:
        json.dump(json_obj["rawData"], file)
        
data = json.load(open('Covid.json'))
df = pd.json_normalize(data)

In [143]:
### User input for converting JSON File to SQL or CSV
print("Enter 'CSV' or 'SQL' to convert JSON file: ")
file_type = input()

Enter 'CSV' or 'SQL' to convert JSON file: 
SQL


In [144]:
def create_sql_file():
    sqlEngine = create_engine("sqlite:///covid_data.db")
    connection = sqlEngine.connect()
    df.to_sql("covid_table", con=connection)

In [145]:
### Convert JSON to CSV file or SQL Database, and save locally
if file_type == "CSV":
    try:
        df.to_csv('Covid_csv.csv')
    except Exception as E_CSV:
        print("Error converting to CSV file: ", E_CSV)
    else:
        print("JSON successfully converted to CSV file")
elif file_type == "SQL":
    try:
        create_sql_file()
    except Exception as E_SQL:
        print("Error converting to SQL Database: ", E_SQL)
    else: print("JSON successfully converted to SQL Database")
else:
    print("Error: Inputted file type is not 'CSV' or 'SQL'")
    

JSON successfully converted to SQL Database


In [146]:
### Summary of data file ingestion
print("Number of records: " + str(df.shape[0]))
print("Number of columns: " + str(df.shape[1]))

df.info()

Number of records: 4016
Number of records: 14
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4016 entries, 0 to 4015
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Active               4016 non-null   object
 1   Admin2               4016 non-null   object
 2   Case_Fatality_Ratio  4016 non-null   object
 3   Combined_Key         4016 non-null   object
 4   Confirmed            4016 non-null   object
 5   Country_Region       4016 non-null   object
 6   Deaths               4016 non-null   object
 7   FIPS                 4016 non-null   object
 8   Incident_Rate        4016 non-null   object
 9   Last_Update          4016 non-null   object
 10  Lat                  4016 non-null   object
 11  Long_                4016 non-null   object
 12  Province_State       4016 non-null   object
 13  Recovered            4016 non-null   object
dtypes: object(14)
memory usage: 439.4+ KB
